In [1]:
import joblib
import numpy as np

import mujoco

from hydrax.algs import CEM, MPPI, PredictiveSampling, Evosax, xNES, CMAES, SVES, GaussianSmoothing, xMPPI, predictive_sampling
from hydrax.simulation.deterministic import run_interactive
from hydrax.tasks.cart_pole import CartPole
from hydrax.tasks.cube import CubeRotation
from hydrax.tasks.pusht import PushT
from hydrax.tasks.humanoid_standup import HumanoidStandup
from hydrax.simulation.traj_opt import traj_opt_helper

In [2]:
# CubeRotation
task = CubeRotation()
mj_model = task.mj_model
mj_model.opt.timestep = 0.01
mj_model.opt.iterations = 10
mj_data = mujoco.MjData(mj_model)

# CartPole
# task = CartPole()
# mj_model = task.mj_model
# mj_model.opt.timestep = 0.01
# mj_model.opt.iterations = 10
# mj_data = mujoco.MjData(mj_model)

# HumanoidStandup
# task = HumanoidStandup()
# mj_model = task.mj_model
# mj_model.opt.timestep = 0.01
# mj_model.opt.o_solimp = [0.9, 0.95, 0.001, 0.5, 2]
# mj_model.opt.enableflags = mujoco.mjtEnableBit.mjENBL_OVERRIDE

# # Set the initial state so the robot falls and needs to stand back up
# mj_data = mujoco.MjData(mj_model)
# mj_data.qpos[:] = mj_model.keyframe("stand").qpos
# mj_data.qpos[3:7] = [0.7, 0.0, -0.7, 0.0]


# PushT
# task = PushT()
# mj_model = task.mj_model
# mj_model.opt.timestep = 0.01
# mj_model.opt.iterations = 10
# mj_data = mujoco.MjData(mj_model)
# mj_data.qpos = [0.1, 0.1, 1.3, 0.0, 0.0]

# Parameters
num_trails = 3
max_iterations = 100
num_samples = 1024
sigma = 0.2
temperature = 0.01
spline = "cubic"
horizon = 0.6
num_knots = 10

In [3]:

ps = PredictiveSampling(
    task,
    num_samples = num_samples,
    noise_level=sigma,
    plan_horizon= horizon,
    spline_type= spline,
    num_knots= num_knots,
)

to = traj_opt_helper(ps, mj_model, mj_data)
to.trails(max_iteration=max_iterations, num_trails = num_trails)


Trajectory Optimization with 10 steps over a 0.6 second horizon.
Jitting the controller...


/home/rooh/miniconda3/envs/hydrax/lib/python3.12/site-packages/jax/_src/interpreters/xla.py:119: RuntimeWarning: overflow encountered in cast
  return np.asarray(x, dtypes.canonicalize_dtype(x.dtype))


Time to jit: 23.054 seconds
path created: data/CubeRotation


100%|█████████████████████████████████████████| 100/100 [00:50<00:00,  1.98it/s]

Optimization done.


ValueError: too many values to unpack (expected 2)

In [ ]:

xmppi = xMPPI(
    task,
    num_samples = num_samples,
    temperature= temperature,
    sigma = sigma,
    lr_sigma=0.5,
    lr_B = 0.05,
    plan_horizon= horizon,
    spline_type=spline,
    num_knots= num_knots
)
to = traj_opt_helper(xmppi, mj_model, mj_data)
to.trails(max_iteration=max_iterations, num_trails = num_trails)


 55%|███████████████████████                   | 55/100 [00:30<00:24,  1.84it/s]

 


🔥fitness = 
[0.62275124 0.2665606  0.30515364 ... 0.3222708  0.31809086 1.2462213 ] 
 🔥any NaN? = False 
 🔥weights (softmax) =
[4.0641389e-04 1.1253745e-03 1.0077923e-03 ... 9.5965335e-04 9.7119203e-04
 6.8347734e-05] 
 🔥any NaN? = False 
 mean = 
 [ 0.302452   -0.28164256 -0.08658618  0.01253819 -0.27224782 -0.41754073
  0.47374785 -0.18197083 -0.2489087   0.06934991 -0.06569049  0.00991528
 -0.00534631 -0.22244743  0.0509695   0.04030607  0.12879269  0.19042405
  0.41250503  0.5447759   0.63264745 -0.03256713  0.96529496 -0.34742996
 -0.16459481  0.22990751  0.11722405 -0.04717013 -0.11890425  0.11635535
 -0.06274844  0.36281046  0.6167413   0.34708902  0.6369097   0.5550032
  0.4442142  -0.75868857  0.6188086  -0.35675752  0.03579503 -0.08984208
 -0.17420284  0.16446973  0.041083    0.08719216  0.31405714  0.02929875
  0.9640246  -0.06954958  0.77765316  0.17204052  0.8028698   0.02259768
  0.678809   -0.2241745  -0.1157406  -0.2586155  -0.04092957 -0.09827401
  0.36639982  0.25

 56%|███████████████████████▌                  | 56/100 [00:31<00:23,  1.85it/s]

 


🔥fitness = 
[0.21316072 0.09778109 1.169998   ... 0.42123234 0.27634808 0.32719052] 
 🔥any NaN? = False 
 🔥weights (softmax) =
[0.00128148 0.00168598 0.00013174 ... 0.00078138 0.00110272 0.00097717] 
 🔥any NaN? = False 
 mean = 
 [ 0.29933617 -0.29183224 -0.08270554  0.01534165 -0.27565035 -0.42668605
  0.47836232 -0.1786648  -0.2437564   0.05980593 -0.07028292  0.02759743
 -0.01755832 -0.21884543  0.03771919  0.02708031  0.1091636   0.1779136
  0.42561045  0.54249763  0.63487977 -0.02922309  0.95340824 -0.34731463
 -0.18645126  0.22552557  0.1188052  -0.03686923 -0.11343615  0.1099746
 -0.06418198  0.37103373  0.6093297   0.3535546   0.643308    0.5421711
  0.44029808 -0.76593953  0.6406758  -0.36603922  0.0390888  -0.10500839
 -0.16304912  0.16329437  0.03911992  0.09379976  0.31987232  0.02724982
  0.963348   -0.08343734  0.77958745  0.18362214  0.7980436   0.01406683
  0.683285   -0.22859062 -0.1005341  -0.26568806 -0.0362308  -0.09219377
  0.38764983  0.2675577  -0.35582164 -0

 57%|███████████████████████▉                  | 57/100 [00:31<00:23,  1.86it/s]

...

 


🔥fitness = 
[0.13494587 0.7639142  0.7377243  ... 0.3976594  0.42044437 0.38716683] 
 🔥any NaN? = False 
 🔥weights (softmax) =
[0.00210834 0.00033624 0.00036296 ... 0.00097932 0.00091631 0.00100977] 
 🔥any NaN? = False 
 mean = 
 [ 0.22018     0.19439639 -0.18137394  0.03155893 -0.13510366  0.12206677
  0.2730349   0.22834636  0.06560762 -0.11598613 -0.26944128 -0.27387238
 -0.03783631  0.07556895 -0.09463698 -0.0294537   0.04764287 -0.4076052
  0.56700504  0.12467236  0.59734213  0.04467468  0.4963375  -0.29488117
 -0.04490602 -0.19052051  0.02815308  0.05779357 -0.07476964 -0.21530783
 -0.04800996  0.27867463  0.45879328  0.04237352  0.71919006  0.59150475
  0.38521177 -0.29833764  0.513236   -0.21883206 -0.00895945  0.00554565
 -0.2765548  -0.10066501  0.17023797  0.08704823 -0.1938352  -0.10736249
  0.5447427  -0.12749295  0.5489098   0.6570666   0.88549995 -0.04408505
  0.62235004  0.14910479  0.12803407 -0.12773748 -0.00664647  0.17612104
  0.30679962 -0.13056877  0.08360547 

 51%|█████████████████████▍                    | 51/100 [00:27<00:29,  1.68it/s]

 


🔥fitness = 
[0.6937967  1.0362583  0.3783158  ... 0.49886566 0.31494248 0.4894039 ] 
 🔥any NaN? = False 
 🔥weights (softmax) =
[0.00035143 0.00011338 0.00099638 ... 0.00066909 0.0012284  0.00069034] 
 🔥any NaN? = False 
 mean = 
 [ 0.22470738  0.17470083 -0.16708466  0.01719309 -0.13592839  0.1190144
  0.28915796  0.22748299  0.05304312 -0.12675475 -0.2567095  -0.29133305
 -0.04557561  0.05694518 -0.09848568 -0.04078289  0.03432018 -0.3957332
  0.56745696  0.11909883  0.5987809   0.03771757  0.5260462  -0.29820785
 -0.02332022 -0.1895715   0.04166428  0.03937065 -0.05821927 -0.21182229
 -0.0476878   0.26623148  0.44861558  0.03357022  0.7081462   0.59213233
  0.38393167 -0.3063161   0.51615    -0.22829035 -0.0225397   0.0020946
 -0.278026   -0.1394214   0.18282753  0.07599914 -0.18823928 -0.09612893
  0.5612471  -0.12219753  0.5420101   0.6311428   0.90679723 -0.07941398
  0.61244756  0.14053856  0.11898343 -0.1308362  -0.00696204  0.17733933
  0.30697793 -0.11403126  0.08740852 -0

 52%|█████████████████████▊                    | 52/100 [00:28<00:27,  1.73it/s]

 


🔥fitness = 
[0.596172   0.3519233  0.59922415 ... 0.5549704  0.42092463 0.49055326] 
 🔥any NaN? = False 
 🔥weights (softmax) =
[0.00052309 0.00105816 0.0005185  ... 0.0005891  0.00086718 0.00070939] 
 🔥any NaN? = False 
 mean = 
 [ 0.21580535  0.16811386 -0.17311932  0.02143108 -0.14639601  0.10191511
  0.30121875  0.22023404  0.0333847  -0.12716064 -0.25829673 -0.2888586
 -0.05189389  0.06802221 -0.10329623 -0.0416856   0.02621396 -0.4132426
  0.5651051   0.11747293  0.5994404   0.01172139  0.5334469  -0.26986152
 -0.02723203 -0.20713504  0.03512046  0.02521827 -0.07440082 -0.20155072
 -0.04310343  0.26923543  0.44835508  0.03835156  0.7197707   0.60126686
  0.38310492 -0.31477565  0.5281211  -0.2397368  -0.0284497   0.02938214
 -0.2813633  -0.12738885  0.18732537  0.06826535 -0.2112911  -0.07933711
  0.55625683 -0.1286811   0.560417    0.62940687  0.9182628  -0.08688004
  0.6186387   0.1416819   0.11905573 -0.14283806  0.01464877  0.17587636
  0.31145015 -0.10580882  0.08086868 -

In [ ]:

xnes = xNES(
    task,
    num_samples = num_samples,
    temperature= temperature,
    sigma = sigma,
    lr_sigma=0.5,
    lr_B = 0.05,
    plan_horizon= horizon,
    spline_type=spline,
    num_knots= num_knots
)
to = traj_opt_helper(xnes, mj_model, mj_data)
to.trails(max_iteration=max_iterations, num_trails = num_trails)


In [ ]:

cem = CEM(
    task,
    num_samples = num_samples,
    num_elites = 10,
    sigma_start= sigma,
    sigma_min = 0.2,
    plan_horizon=1.0,
    spline_type=spline,
    num_knots=num_knots,
)
to = traj_opt_helper(cem, mj_model, mj_data)
to.trails(max_iteration=max_iterations, num_trails = num_trails)


In [ ]:

cmaes = CMAES(
    task,
    num_samples = num_samples,
    sigma = sigma,
    plan_horizon= horizon,
    spline_type=spline,
    num_knots=num_knots,
)

to = traj_opt_helper(cmaes, mj_model, mj_data)
to.trails(max_iteration=max_iterations, num_trails = num_trails)


In [ ]:
mppi = MPPI(
    task,
    num_samples = num_samples,
    temperature = temperature,
    noise_level= sigma,
    plan_horizon= horizon,
    spline_type=spline,
    num_knots=num_knots
)
mj_model = task.mj_model

to = traj_opt_helper(mppi, mj_model, mj_data)
to.trails(max_iteration=max_iterations, num_trails = num_trails)


In [ ]:
gs = GaussianSmoothing(
    task,
    num_samples = num_samples,
    temperature = temperature,
    sigma = sigma,
    plan_horizon= horizon,
    spline_type= spline,
    num_knots=num_knots
)
mj_model = task.mj_model

to = traj_opt_helper(gs, mj_model, mj_data)
to.trails(max_iteration=max_iterations, num_trails = num_trails)


In [ ]:
task_name = task.__class__.__name__
path = "data/" + task_name + "/"

PS_costs = joblib.load(path + "PredictiveSampling_costs_trails_average.pkl")
xNES_costs = joblib.load(path + "xNES_costs_trails_average.pkl")
MPPI_costs = joblib.load(path + "MPPI_costs_trails_average.pkl")
xMPPI_costs = joblib.load(path + "xMPPI_costs_trails_average.pkl")
CMAES_costs = joblib.load(path + "CMAES_costs_trails_average.pkl")
CEM_costs = joblib.load(path + "CEM_costs_trails_average.pkl")
GS_costs = joblib.load(path + "GaussianSmoothing_costs_trails_average.pkl")


In [ ]:
import matplotlib.pyplot as plt

# print(xNES_costs)
# # Plot the costs
plt.figure(figsize=(10, 6))

xNES_costs = np.array(xNES_costs).T
MPPI_costs = np.array(MPPI_costs).T
xMPPI_costs = np.array(xMPPI_costs).T
CMAES_costs = np.array(CMAES_costs).T
CEM_costs = np.array(CEM_costs).T
GS_costs = np.array(GS_costs).T
PS_costs = np.array(PS_costs).T

plt.plot(xNES_costs,   label="xNES")
plt.plot(MPPI_costs,   label="MPPI")
plt.plot(xMPPI_costs,   label="xMPPI")
plt.plot(CMAES_costs,  label="CMA-ES")
plt.plot(CEM_costs,  label="CEM")
plt.plot(GS_costs,  label="Randomized smoothing")
plt.plot(PS_costs,  label="Predictive sampling")



plt.title( task_name + " Task")
plt.xlabel("Iteration")
plt.ylabel("Cost")

# single legend call:
plt.legend(loc="best")
plt.tight_layout()
plt.show()
